# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig, Environment
from azureml.widgets import RunDetails
from azureml.core.dataset import Dataset
import os
import joblib
import pickle
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

# Packages for creating the compute cluster 
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

## Dataset
~~TODO:~~
**DONE:** Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.
> This portion has just simply been handled in the `training_hyperdrive` area. The data is located in workspace and loaded to a pandas dataframe; later in our **HyperDrive Config** section, it's passed as an argument to be used in ``training_hyperdrive.py``

In [2]:
ws = Workspace.from_config()

# Create experiment.
experiment_name = 'predict-housing-prices-hyperdrive'

experiment=Experiment(ws, experiment_name)

# Get the datastore
datastore = ws.get_default_datastore()


# Create cluster
cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f'Using existing cluster with name {cluster_name}.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4,
                                                           min_nodes=1)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
key = 'AmesHousingData'
if key in ws.datasets.keys():
    data = ws.datasets[key]
    print(f"Dataset {key} located and loaded in.")
else:
    print(f"Cannot find {key}.")

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ETUREFHLX to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded...............
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
Dataset AmesHousingData located and loaded in.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# Create early termination policy to cancel the run if criteria is met when comparing to the current best score.
early_termination_policy = BanditPolicy(slack_factor=0.01, delay_evaluation=30)


# Training parameters.
param_sampling = RandomParameterSampling(
    {
        "--learning_rate": uniform(0.01, 0.03),
        "--n_estimators": choice(100, 150, 200, 250) 
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=".", compute_target=compute_target, entry_script="training_hyperdrive.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling = param_sampling,
                                         policy = early_termination_policy,
                                         estimator=estimator,
                                         primary_metric_name = 'normalized_root_mean_squared_error',
                                         primary_metric_goal = PrimaryMetricGoal.MINIMIZE,
                                         max_concurrent_runs=3,
                                         max_total_runs=12,
                                         max_duration_minutes=30 
                                         )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
# Submit experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details
The output of run details is shown below.

In [6]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
# Get the best run
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
best_run.get_details()

['--learning_rate', '0.011086610775551951', '--n_estimators', '150']


{'runId': 'HD_e8466a5e-90cc-451a-82db-bf4fec11f0a9_4',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T17:09:58.424077Z',
 'endTimeUtc': '2021-04-19T17:10:36.011058Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '6983863f-6367-42be-acf0-677985a33f25',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '274ea3a5-a4bc-4b6a-9d83-72fb98e7d4e4'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'training_hyperdrive.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--learning_rate',
   '0.011086610775551951',
   '--n_estimators',
   '150'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'capstone-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSe

In [50]:
#TODO: Save the best model
# get run
run = Run(experiment,'HD_e8466a5e-90cc-451a-82db-bf4fec11f0a9_4')

# get best one
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: HD_2c71ec54-7f0b-408a-a967-21dbc464c855_2
Web View: https://ml.azure.com/runs/HD_2c71ec54-7f0b-408a-a967-21dbc464c855_2?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-143099/workspaces/quick-starts-ws-143099&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_2c71ec54-7f0b-408a-a967-21dbc464c855_2
Web View: https://ml.azure.com/runs/HD_2c71ec54-7f0b-408a-a967-21dbc464c855_2?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-143099/workspaces/quick-starts-ws-143099&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_2c71ec54-7f0b-408a-a967-21dbc464c855_2',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T04:06:43.111156Z',
 'endTimeUtc': '2021-04-19T04:07:46.401709Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'f4a00de1-c92d-4556-8c4a-cd603962cc40',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'b3e7f477-8026-4fec-bad7-e272c7f4ea9b'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'training_hyperdrive.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--learning_rate',
   '0.010193825887921974',
   '--n_estimators',
   '100'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'capstone-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSe

In [51]:
# Get the best run as well as the model and print the details.
best_run, model = run.get_output()
print(f"Best Run:\t{best_run}\n\nBest Model:\t{model}")

AttributeError: 'Run' object has no attribute 'get_output'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service